In [1]:
!pip install aicsimageio[all] --user
!pip install bioformats_jar

In [2]:
!pip install aicspylibczi>=3.1.1 fsspec>=2022.8.0

In [3]:
!pip install opencv-python

In [4]:
from aicsimageio.writers import OmeTiffWriter
from aicsimageio import AICSImage
from skimage.io import imshow
from tqdm import tqdm
import os
import cv2

import numpy as np

C:\Users\schatzm\Anaconda3\envs\czi_processing\lib\site-packages\pydantic\_migration.py:276: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [5]:
from aicspylibczi import CziFile

In [6]:
# %%time
# # Path to your CZI file
# czi_file_path = '../../test/2023_05_22_Pyl-OB_010_small.czi'
# name = "output"

# # czi_file_path = '../../230518_ML_pyl_OB/2023_05_21_Pyl-OB_001.czi'
# # name = "2023_05_21_Pyl-OB_001"

# czi = CziFile(czi_file_path)

# # Get the shape of the data, the coordinate pairs are (start index, size)
# # dimensions = czi.dims_shape()  #  [{'X': (0, 1900), 'Y': (0, 1300), 'Z': (0, 60), 'C': (0, 4), 'S': (0, 40), 'B': (0, 1)}]

# print(czi.dims)

# print(czi.size)  #  (1, 40, 4, 60, 1300, 1900)
# size=czi.size

# czi.is_mosaic()



# output_folder = name+'/'

# if not os.path.exists(output_folder):
#         os.makedirs(output_folder)
        
        

# # czi.get_dims_shape()
# #  working but slow
# mosaic_data = czi.read_mosaic(C=0, scale_factor=1, Z=0)
# mosaic_data.shape

# mosaic_data_shape = mosaic_data.shape

# subpart_size = [1600,1920]
# # Calculate the number of subparts in each dimension
# num_subparts_x = mosaic_data_shape[2] // subpart_size[0]
# num_subparts_y = mosaic_data_shape[3] // subpart_size[1]
    
# # Calculate the total number of iterations
# total_iterations = num_subparts_x * num_subparts_y * size[2]

# # Initialize the tqdm progress bar
# progress_bar = tqdm(total=total_iterations, desc=f"Processing {name}")

# for z in range(size[2]):
#     #read Z stack
#     mosaic_data = czi.read_mosaic(C=0, scale_factor=1, Z=z)
    
#     for x in range(num_subparts_x):
#         for y in range(num_subparts_y):       
#             # Extract the subpart
#             subpart = mosaic_data[:, :, x * subpart_size[0]:(x + 1) * subpart_size[0],
#                                   y * subpart_size[1]:(y + 1) * subpart_size[1], :]

#             # Create an AICSImage object
#             aics_image = AICSImage(subpart)

#             # Define the output file path
#             output_file = os.path.join(output_folder, f"{name}_subpart_z-{z}_x-{x}_y-{y}.ome.tiff")

#             # Save the subpart as an OME-TIFF
#             # aics_image.save(output_file)

#             # container = cv2.cvtColor(container, cv2.COLOR_BGR2RGB)
#             # Save the subpart as a TIFF file
#             OmeTiffWriter.save(cv2.cvtColor(np.squeeze(subpart), cv2.COLOR_BGR2RGB), output_file, dim_order="YXS")
            
#             # Update the progress bar
#             progress_bar.update(1)
            
# # Close the progress bar when the loops are done
# progress_bar.close()

In [7]:
# Append Row to DataFrame
# list_row = {"X": 27000, "Y": 2000, "Z": 2000, "focus": 2000}
# df.loc[len(df)]= list_row

In [16]:
from aicsimageio.writers import OmeTiffWriter
from aicsimageio import AICSImage
from tqdm import tqdm
import os
import cv2
import pandas as pd


def process_czi_file(czi_file_path, output_folder, subpart_size):
    df = pd.DataFrame(columns=('X', 'Y', 'Z', 'focus'))
    
#     # read metadata
#     # Open the CZI file using aicsimageio
#     img = AICSImage(czi_file_path)

#     # Read the metadata
#     metadata = img.metadata
    
#     # define scaling from metadata or use defualt scaling
#     try:
#         pixels_physical_size = [metadata['XScale'],
#                                 metadata['YScale'],
#                                 metadata['ZScale']]
#     except Exception as error:
#         print(error.__class__.__name__ + ": " + error.msg)
    # Create a CziFile object
    czi = CziFile(czi_file_path)

    df = pd.DataFrame(columns=['X','Y','Z','focus'])
    
    # Get the shape of the data
    size = czi.size
    
    print(czi.dims)
    print(size)

    # Check if the file is a mosaic
    is_mosaic = czi.is_mosaic()

    if not is_mosaic:
        print("The provided CZI file is not a mosaic.")
        return

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    mosaic_data_shape = czi.read_mosaic(C=0, scale_factor=1, Z=0).shape
    num_subparts_x = mosaic_data_shape[2] // subpart_size[0]
    num_subparts_y = mosaic_data_shape[3] // subpart_size[1]
    total_iterations = num_subparts_x * num_subparts_y * size[2]
    
    print(f"Total integrations: X {num_subparts_x}, Y {num_subparts_y}, Z {size[2]}")

    progress_bar = tqdm(total=total_iterations, desc=f"Processing {os.path.basename(czi_file_path)}")

    for z in range(size[2]):
        mosaic_data = czi.read_mosaic(C=0, scale_factor=1, Z=z)

        for x in range(num_subparts_x):
            for y in range(num_subparts_y):
                subpart = mosaic_data[:, :, x * subpart_size[0]:(x + 1) * subpart_size[0],
                                      y * subpart_size[1]:(y + 1) * subpart_size[1], :]

                aics_image = AICSImage(subpart)
                
                # Create a CziFile object

                output_file = os.path.join(output_folder, f"subpart_z-{z}_x-{x}_y-{y}.ome.tiff")
                
                focus = cv2.Laplacian(np.squeeze(subpart), cv2.CV_64F, ksize=5).var()
                # Append Row to DataFrame
                list_row = {"X": x, "Y": y, "Z": z, "focus": focus}
                df.loc[len(df)]= list_row
                
                try:
                    # without metadata
                    OmeTiffWriter.save(cv2.cvtColor(np.squeeze(subpart), cv2.COLOR_BGR2RGB), output_file, dim_order="YXS")
                    
                # Create an OmeTiffWriter instance
                
                    # OmeTiffWriter.save(cv2.cvtColor(np.squeeze(subpart), cv2.COLOR_BGR2RGB), output_file, dim_order="YXS", pixels_physical_size=pixels_physical_size)
                except Exception as error:
                    print(error.__class__.__name__ + ": " + error.msg)
                    
                progress_bar.update(1)

    progress_bar.close()
    output_csv = os.path.join(output_folder, f"_focus_estimate.csv")
    df.to_csv(output_csv)


In [17]:
%%time
# # czi_file_path = '../../230518_ML_pyl_OB/2023_05_21_Pyl-OB_001.czi'
# # name = "2023_05_21_Pyl-OB_001"

# # test
czi_file_path = '../../test/2023_05_22_Pyl-OB_010_small.czi'
output_folder = "output"

# # Usage example:
# # czi_file_path = '../../230518_ML_pyl_OB/2023_05_22_Pyl-OB_024.czi'
# # output_folder = "2023_05_22_Pyl-OB_024"

# # czi_file_path = '../../230518_ML_pyl_OB/2023_05_22_Pyl-OB_023.czi'
# # output_folder = "2023_05_22_Pyl-OB_023"

subpart_size = [1600, 1920]
process_czi_file(czi_file_path, output_folder,subpart_size)

SCZMYXA
(1, 1, 49, 42, 666, 872, 3)
Total integrations: X 3, Y 4, Z 49


Processing 2023_05_22_Pyl-OB_010_small.czi: 100%|████████████████████████████████████| 588/588 [04:43<00:00,  2.07it/s]

CPU times: total: 11min 29s
Wall time: 4min 45s


In [ ]:
%%time
files = os.listdir("../../230518_ML_pyl_OB/")
for file in files:

    # Get the file extension
    file_extension = os.path.splitext(file)[1]
    file_w = os.path.splitext(file)[0]

    # Check if the file extension is ".czi"
    if file_extension.lower() == ".czi":
        print(file_w)
        fpath = f'../../230518_ML_pyl_OB/{file}'
        print(fpath)
        subpart_size = [1600, 1920]
        process_czi_file(fpath, file_w,subpart_size)

2023_05_18_Pyl-OB_001
../../230518_ML_pyl_OB/2023_05_18_Pyl-OB_001.czi
SCZMYXA
(1, 1, 49, 1267, 1200, 1600, 3)
Total integrations: X 16, Y 54, Z 49


Processing 2023_05_18_Pyl-OB_001.czi: 100%|████████████████████████████████████| 42336/42336 [7:29:20<00:00,  1.57it/s]


2023_05_21_Pyl-OB_001
../../230518_ML_pyl_OB/2023_05_21_Pyl-OB_001.czi
SCZMYXA
(1, 1, 49, 1121, 1200, 1600, 3)
Total integrations: X 15, Y 54, Z 49


Processing 2023_05_21_Pyl-OB_001.czi:  46%|███████████████▍                  | 18089/39690 [3:11:19<1:45:06,  3.43it/s]

In [ ]:
xxx

In [14]:
 czi_file_path = '../../230518_ML_pyl_OB/2023_05_22_Pyl-OB_024.czi'
img = AICSImage(czi_file_path)

    # Read the metadata
metadata = img.metadata

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
import aicsimageio

In [ ]:
czi2 = aicsimageio.readers.czi_reader.CziReader(czi_file_path)
met = czi2.metadata